# IG03 - Medicamentos no cobrados a pacientes VSC

El indicador IG04 verifica los montos asociados a medicamentos que no han sido cobrados a los pacientes VSC. El riesgo que busca mitigar es el tener pérdidas financieras por prestaciones entregadas y no cobradas al paciente VSC. El área responsable es la Gerencia de Operaciones HT.

El siguiente código es para ejecutar el indicador de Auditoría Continua IG03 - Medicamentos no cobrados a pacientes VSC.

 - ***Fecha creación: 18.08.2020***
 - ***Fecha modificación_1: 21.08.2020***
 - ***Fecha modificación_2: 02.09.2020***

## Importar librerías
---

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Utilidades 

In [2]:
def clean_blankspace(df):
    #eliminar espacios en nombres de columnas
    columns = list(df.columns)
    columns = [w.replace(' ','') for w in columns]
    df.columns = columns
    #Eliminar espacios en cada celda del dataframe
    for columna in list(df.columns):
        if df.dtypes[columna] == np.dtype('O'):
            df[columna] = df[columna].str.replace(' ','')
    return df

def clean_columns(txt):
    df = txt.drop(columns=txt.columns[0:2]).drop(columns=txt.columns[-1]).dropna(how='all').reset_index(drop=True)
    return df

def num_format(df,col):
    df[col] = df[col].str.replace('.','').str.replace(',','.').astype(float)
    if (col=='DMBTR') or (col=='ERFMG') or (col=='VERPR'):
        df[col] = df[col]*100
    return df

## Cargar tablas
---

In [3]:
n1meevent = pd.read_csv('N1MEEVENT.txt', sep='|', header=3, encoding='latin1',low_memory=False)
nfal = pd.read_csv('NFAL.txt', sep='|', header=3, encoding='latin1',low_memory=False)
n1formulary = pd.read_csv('N1FORMULARY.txt', sep='|', header=3, encoding='latin1',low_memory=False)
mara = pd.read_csv('MARA.txt', sep='|', header=3, encoding='latin1',low_memory=False)
mbew = pd.read_csv('MBEW.txt', sep='|', header=3, encoding='latin1',low_memory=False)
n1odrug = pd.read_csv('N1ODRUG.txt', sep='|', header=3, encoding='latin1',low_memory=False)
n1meorder = pd.read_csv('N1MEORDER.txt', sep='|', header=3, encoding='latin1',low_memory=False)

## Limpieza

#### Eliminar filas y columnas Nan

In [4]:
n1meevent = clean_columns(n1meevent)
nfal = clean_columns(nfal)
n1formulary = clean_columns(n1formulary)
mara = clean_columns(mara)
mbew = clean_columns(mbew)
n1odrug = clean_columns(n1odrug)
n1meorder = clean_columns(n1meorder)

#### Eliminar espacios en nombres de columnas y cada celda

In [5]:
n1meevent = clean_blankspace(n1meevent)
nfal = clean_blankspace(nfal)
n1formulary = clean_blankspace(n1formulary)
mara = clean_blankspace(mara)
mbew = clean_blankspace(mbew)
n1odrug = clean_blankspace(n1odrug)
n1meorder = clean_blankspace(n1meorder)

#### Cambio de formato de columnas

In [6]:
mbew = num_format(mbew,'VERPR')
nfal['FALNR'] = nfal['FALNR'].str.replace('R','')    
nfal = nfal.replace(r'^\s*$', np.nan, regex=True)                  # reemplaza valores vacios con np.nan
n1formulary = n1formulary.replace(r'^\s*$', np.nan, regex=True)    # reemplaza valores vacios con np.nan

#### Establecer esquema de datos

In [7]:
schema_n1meevent = {'MEORDID': 'float64',  # N_prescripcion 
                    'FALNR': 'float64',    # Episodio 
                    'MESID': 'float64',    # Desc_evento 
                    'MASTEV': 'float64',   # N_evento 
                    'ERDAT': str}          # Fecha_creacion

schema_nfal = {'FALNR': 'float64',         # Episodio 
                'FALAR': 'float64',        # Clase_episodio 
                'FATYP': 'float64'}        # Tipo_episodio

schema_n1formulary = {'DRUGID': str,       # ID_medicamento
                      'MATNR': 'float64'}  # Material

schema_mara = {'MATNR': 'float64',         # Material
               'MTART': str}               # Tipo_material

schema_mbew = {'MATNR': 'float64',         # Material
               'VERPR': 'float64'}         # Precio_variable

schema_n1odrug = {'MEORDID': 'float64',    # N_prescripcion 
                  'DRUGID': str}           # ID_medicamento

schema_n1meorder = {'MEORDID': 'float64',  # N_prescripcion 
                'PATNR': 'float64',        # Paciente 
                'FALNR': 'float64',        # Episodio 
                'MOTX': str,               # Texto_presc 
                'ORGPF': str,              # UO_UE 
                'MOTYPID': 'float64'}      # Tipo_presc

n1meevent = n1meevent.astype(schema_n1meevent)
nfal = nfal.astype(schema_nfal)
n1formulary = n1formulary.astype(schema_n1formulary)
mara = mara.astype(schema_mara)
mbew = mbew.astype(schema_mbew)
n1odrug = n1odrug.astype(schema_n1odrug)
n1meorder = n1meorder.astype(schema_n1meorder)

### Filtrar tablas

In [8]:
#Filtrar tipos de episodio y clase de episodio a analizar

nfal = nfal.loc[(nfal['FALAR']==1) & (nfal['FATYP']==2)].reset_index(drop=True)

#Filtrar tipos de prescripción a analizar

n1meorder = n1meorder.loc[(n1meorder['MOTYPID']==1)].reset_index(drop=True)

### Eliminar columnas innecesarias

In [9]:
#Eliminamos la columna "Clase_episodio" porque no la utilizaremos más adelante

nfal = nfal.drop(['FALAR'], axis=1)

#Eliminamos la columna "Tipo_presc" porque no la utilizaremos más adelante

n1meorder = n1meorder.drop(['MOTYPID'], axis=1)


## Ejecutar indicador
---

Para poder llegar a la información necesaria, en necesario hacer los siguientes cruces en orden:
 - **n1meveent y nfal** por `Episodio` >> parte1
 - **parte1 y n1odrug** por `N_prescripcion` >> aux1
 - **aux1 y n1formulary** por `ID_medicamento` >> aux2
 - **aux2 y mara** por `Material` >> aux3
 - **aux3 y mbew** por `Material` >> aux4
 - **aux4 y n1meorder** por `N_prescripcion` >> IG04

### n1meveent y nfal

In [10]:
parte1 = pd.merge(n1meevent, nfal, on = 'FALNR', how='inner')
parte1.head()

,MEORDID,FALNR,MESID,MASTEV,ERDAT,FATYP
0,8106830.0,5017332.0,3.0,0.0,03.07.2020,2.0
1,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0
2,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0
3,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0
4,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0


### Parte1 y n1odrug

In [11]:
aux1 = pd.merge(parte1, n1odrug, on='MEORDID', how='inner')
aux1

,MEORDID,FALNR,MESID,MASTEV,ERDAT,FATYP,DRUGID
0,8106830.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8B587EC9976E77
1,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62
2,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62
3,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62
4,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62
...,...,...,...,...,...,...,...
1890,8266634.0,5020377.0,3.0,0.0,20.08.2020,2.0,0050568370071ED7B8B26A868B570E7D
1891,8270397.0,5093919.0,3.0,0.0,20.08.2020,2.0,00505683406F1EE68FD97291AA064E62
1892,8270401.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547690F9
1893,8270402.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547850F9


### aux1 y n1formulary

In [12]:
aux2 = pd.merge(aux1, n1formulary, on='DRUGID', how='left')
aux2.head()

,MEORDID,FALNR,MESID,MASTEV,ERDAT,FATYP,DRUGID,MATNR
0,8106830.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8B587EC9976E77,251533.0
1,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0
2,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0
3,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0
4,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0


### aux2 y mara

In [13]:
aux3 = pd.merge(aux2, mara, on='MATNR', how='left')
aux3

,MEORDID,FALNR,MESID,MASTEV,ERDAT,FATYP,DRUGID,MATNR,MTART
0,8106830.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8B587EC9976E77,251533.0,Z1ME
1,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME
2,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME
3,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME
4,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME
...,...,...,...,...,...,...,...,...,...
1890,8266634.0,5020377.0,3.0,0.0,20.08.2020,2.0,0050568370071ED7B8B26A868B570E7D,253783.0,Z1ME
1891,8270397.0,5093919.0,3.0,0.0,20.08.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME
1892,8270401.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547690F9,251252.0,Z1ME
1893,8270402.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547850F9,251253.0,Z1ME


### aux3 y mbew

In [14]:
aux4 = pd.merge(aux3, mbew, on='MATNR', how='left')
aux4

,MEORDID,FALNR,MESID,MASTEV,ERDAT,FATYP,DRUGID,MATNR,MTART,VERPR
0,8106830.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8B587EC9976E77,251533.0,Z1ME,30.0
1,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0
2,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0
3,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0
4,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0
...,...,...,...,...,...,...,...,...,...,...
1890,8266634.0,5020377.0,3.0,0.0,20.08.2020,2.0,0050568370071ED7B8B26A868B570E7D,253783.0,Z1ME,5486.0
1891,8270397.0,5093919.0,3.0,0.0,20.08.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0
1892,8270401.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547690F9,251252.0,Z1ME,1707.0
1893,8270402.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547850F9,251253.0,Z1ME,8.0


### aux4 y n1meorder

In [15]:
IG04 = pd.merge(aux4, n1meorder, on=['MEORDID','FALNR'], how='inner').drop_duplicates()
IG04

,MEORDID,FALNR,MESID,MASTEV,ERDAT,FATYP,DRUGID,MATNR,MTART,VERPR,PATNR,MOTX,ORGPF
0,8106830.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8B587EC9976E77,251533.0,Z1ME,30.0,1.003507e+09,TENSIOMAX10MG-DOSISUNITARIABIOEQ,HTE5PON
1,8106831.0,5017332.0,3.0,0.0,03.07.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0,1.003507e+09,ZELASTER1GR,HTE5PON
7,8107192.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8CAAC0547850F9,251253.0,Z1ME,8.0,1.003507e+09,PARACETAMOL500MGANDR,HTE5PON
10,8107193.0,5017332.0,3.0,0.0,03.07.2020,2.0,00163E4E71851EE88CED2C03A6A2504E,254066.0,Z1ME,555.0,1.003507e+09,KETOPROFENOEV100MGEUROFARMA,HTE5PON
11,8107205.0,5017332.0,3.0,0.0,03.07.2020,2.0,005056830B101ED4BA8CA647A3EC30F6,251074.0,Z1ME,96.0,1.003507e+09,METAMIZOL1GR/2MLSAND,HTE5PON
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,8266634.0,5020377.0,3.0,0.0,20.08.2020,2.0,0050568370071ED7B8B26A868B570E7D,253783.0,Z1ME,5486.0,1.001063e+09,REMIFENTANILO1MGFK,HTQPBCE
1714,8270397.0,5093919.0,3.0,0.0,20.08.2020,2.0,00505683406F1EE68FD97291AA064E62,253064.0,Z1ME,406.0,1.000410e+09,ZELASTER1GR,HTE4CDT
1715,8270401.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547690F9,251252.0,Z1ME,1707.0,1.000410e+09,PARACETAMOL10MG/ML,HTE4CDT
1716,8270402.0,5093919.0,3.0,0.0,20.08.2020,2.0,005056830B101ED4BA8CAAC0547850F9,251253.0,Z1ME,8.0,1.000410e+09,PARACETAMOL500MGANDR,HTE4CDT


In [16]:
suma_precio = IG04['VERPR'].sum().reshape(1,)
np.shape(suma_precio)

(1,)

In [17]:
prescripciones_sin_duplicado = IG04['MEORDID'].drop_duplicates().count().reshape(1,)
np.shape(prescripciones_sin_duplicado)

(1,)

In [18]:
lista = [suma_precio, prescripciones_sin_duplicado]

In [19]:
Totales = pd.DataFrame({'Suma': lista[0], 'Cantidad': lista[1]})
Totales

,Suma,Cantidad
0,1356898.0,1098


#### Cambio de nombre de columnas

In [20]:
IG04.rename(columns={'MEORDID':'N_prescripcion',
                     'FALNR':'Episodio ',
                     'MESID':'Desc_evento',
                     'MASTEV':'N_evento',
                     'ERDAT':'Fecha_creacion',
                     'FATYP':'Tipo_episodio',
                     'DRUGID':'ID_medicamento',
                     'MATNR':'Material',
                     'MTART':'Tipo_material',
                     'VERPR':'Precio_variable',
                     'PATNR':'Paciente',
                     'MOTX':'Texto_presc',
                     'ORGPF':'UO_UE'},
            inplace = True)

## Guardar en Excel

In [21]:
nombre_archivo = 'IG04 '+datetime.now().strftime("%d-%m-%y_%Hh%Mm")+'.xlsx'
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')

IG04.to_excel(writer, sheet_name='IG04')
Totales.to_excel(writer, sheet_name='Totales')

writer.save()